In [2]:
# import packages 
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
import os
import pandas as pd
import seaborn as sns
import sys
import warnings
from PIL import Image


root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

preprocessed_folder = "/media/ceolin/Data/Lab Gompel/Projects/B_lines_Data/BlineSelected/preprocessed/"
filename = 'DatasetInformation.xlsx'
dataframe = pd.read_excel(os.path.join(preprocessed_folder,filename))
dataframe.head()

,Unnamed: 0,constructs,image file name
0,0,B00,B00_replicate_0.tif
1,0,B00,B00_replicate_1.tif
2,0,B00,B00_replicate_2.tif
3,0,B00,B00_replicate_3.tif
4,0,B00,B00_replicate_4.tif


In [ ]:
def average_phenotype(dataframe, folder, construct=None):
    if construct:
        dataframe = dataframe[dataframe['construct']==construct]

    image_names = dataframe["file name"].values
    
    temp_image = Image.open(os.path.join(folder,image_names[0]))
    image = np.asarray(temp_image)/len(image_names)
    
    for image_file_name in image_names[1:]:
        temp_image = Image.open(os.path.join(folder,image_file_name))
        temp_image = np.asarray(temp_image)/len(image_names)
        image = (image + temp_image)
        
    return image

average_D0 = average_phenotype(dataframe, preprocessed_folder, "D0")

image_names = dataframe["file name"].values
dataframe["distance"] = 0

for image_file_name in image_names[1:]:
    temp_image = np.asarray(Image.open(os.path.join(preprocessed_folder,image_file_name)))
    distance_from_D0 = np.sum(np.power(average_D0 - temp_image,2))
    dataframe.loc[dataframe["file name"]==image_file_name, "distance"] = distance_from_D0

dataframe["boundary"] = 0
dataframe.loc[dataframe["construct"] == "D0", "boundary"] = -5419
dataframe.loc[dataframe["construct"] == "D6", "boundary"] = -2519
dataframe.loc[dataframe["construct"] == "D3", "boundary"] = -3190
dataframe.loc[dataframe["construct"] == "D2", "boundary"] = -3496
dataframe.loc[dataframe["construct"] == "empty", "boundary"] = 0

# normalization

av_dist_empty = dataframe.loc[dataframe["construct"] == "empty", "distance"].mean()
dataframe["distance"] = dataframe["distance"]/av_dist_empty